In [1]:
import pandas as pd
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [33]:
df = pd.DataFrame(pd.read_csv("../logs/arbbot.csv"))
df["humanTimestamp"] = pd.to_datetime(df['blockTimestamp'], unit='s')
df["runTime"] = df["endTimestamp"] - df["startTimestamp"]

In [34]:
# Limit to hour span
hour_limit = 24
sec_limit = hour_limit * 3600
df = df[df.blockTimestamp > time.time()-sec_limit]

In [35]:
# df[df["netProfit"]>0].sort_values(by="netProfit", ascending=0)
under10 = df[(df.netProfit>0)&(df.optimalAmount<10)].sort_values(by="netProfit", ascending=0)
print(f"Amount of opp with <10eth input (24h): {len(under10)}")
print(f"Average input amount (24h): {under10.optimalAmount.mean()} eth")
print(f"Average net profit (24h): {under10.netProfit.mean()} eth")

Amount of opp with <10eth input (24h): 394
Average input amount (24h): 5.176802030456853 eth
Average net profit (24h): 0.06331523652284264 eth


In [36]:
# df[df["netProfit"]>0].sort_values(by="netProfit", ascending=0)
above10 = df[(df.netProfit>0)&(df.optimalAmount>10)&(df.optimalAmount<50)].sort_values(by="netProfit", ascending=0)
print(f"Amount of opp (24h): {len(above10)}")
print(f"Average input amount (24h): {above10.optimalAmount.mean()} eth")
print(f"Average net profit (24h): {above10.netProfit.mean()} eth")

Amount of opp (24h): 419
Average input amount (24h): 25.967231503579953 eth
Average net profit (24h): 0.26633094081145586 eth


## Best opportunities so far

In [37]:
df.sort_values(by="netProfit", ascending=0).head(20)

,blockNumber,blockTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,startTimestamp,endTimestamp,humanTimestamp,runTime
62112,11578667.0,1609639178,weth2yfi2weth_uniswap2Sushiswap,I0009,127.2,4.110,4.084,0.02548,260000,1.609639e+09,1.609639e+09,2021-01-03 01:59:38,0.519222
62116,11578668.0,1609639189,weth2yfi2weth_sushiswap2Uniswap,I0010,123.3,3.937,3.915,0.02158,260000,1.609639e+09,1.609639e+09,2021-01-03 01:59:49,1.504879
62085,11578659.0,1609639057,weth2yfi2weth_sushiswap2Uniswap,I0010,123.3,3.937,3.911,0.02548,260000,1.609639e+09,1.609639e+09,2021-01-03 01:57:37,0.640265
62159,11578678.0,1609639304,weth2yfi2weth_uniswap2Sushiswap,I0009,121.7,3.760,3.742,0.01768,260000,1.609639e+09,1.609639e+09,2021-01-03 02:01:44,0.321398
62000,11578633.0,1609638691,weth2yfi2weth_sushiswap2Uniswap,I0010,119.1,3.666,3.636,0.02982,260000,1.609639e+09,1.609639e+09,2021-01-03 01:51:31,0.342044
62288,11578707.0,1609639699,weth2yfi2weth_uniswap2Sushiswap,I0009,118.3,3.546,3.521,0.02484,260000,1.609640e+09,1.609640e+09,2021-01-03 02:08:19,1.435516
62321,11578714.0,1609639795,weth2yfi2weth_uniswap2Sushiswap,I0009,118.2,3.543,3.517,0.02548,260000,1.609640e+09,1.609640e+09,2021-01-03 02:09:55,0.508792
62230,11578693.0,1609639497,weth2yfi2weth_uniswap2Sushiswap,I0009,118.1,3.537,3.515,0.02132,260000,1.609640e+09,1.609640e+09,2021-01-03 02:04:57,0.434971
62226,11578692.0,1609639485,weth2yfi2weth_uniswap2Sushiswap,I0009,118.1,3.537,3.511,0.02626,260000,1.609639e+09,1.609640e+09,2021-01-03 02:04:45,0.639187
62363,11578727.0,1609639946,weth2yfi2weth_uniswap2Sushiswap,I0009,117.4,3.491,3.467,0.02369,260000,1.609640e+09,1.609640e+09,2021-01-03 02:12:26,2.170566


## Statistics

In [21]:
# Limit opportuninities to the ones beyond the threshold to account for gas costs
net_profitable_opps = df[df.netProfit > 0]
opp_count = net_profitable_opps.blockNumber.count()
runtime_mean = net_profitable_opps.runTime.mean()
max_profit = net_profitable_opps.netProfit.max()
mean_profit = net_profitable_opps.netProfit.mean()
total_profit = net_profitable_opps.netProfit.sum()

In [22]:
print(f"Opportunitys count: {opp_count}")
print(f"Latency average: {runtime_mean:.2f} sec")
print(f"Max profit: {max_profit:.2f} ETH")
print(f"Average profit: {mean_profit:.2f} ETH")

Opportunitys count: 75
Latency average: 1.46 sec
Max profit: 0.19 ETH
Average profit: 0.02 ETH


## Instruction analysis

In [23]:
opp_comp_df = net_profitable_opps.groupby("instrName").agg({"blockNumber": "count", "netProfit": ["max", "sum", "mean"]})
opp_comp_df.columns = ["count", "max_profit", "summed_profit", "mean_profit"]
opp_comp_df.index.name = None
opp_comp_df.sort_values(by="mean_profit", ascending=0)

,count,max_profit,summed_profit,mean_profit
weth2dai2weth_uniswap2Sushiswap,9,0.188600,0.446580,0.049620
weth2band2weth_sushiswap2Uniswap,2,0.089410,0.094066,0.047033
weth2kp3r2weth_uniswap2Sushiswap,9,0.042760,0.241262,0.026807
weth2inj2weth_sushiswap2Uniswap,4,0.031660,0.057360,0.014340
weth2inj2weth_uniswap2Sushiswap,14,0.019500,0.140702,0.010050
weth2yfi2weth_sushiswap2Uniswap,2,0.016610,0.018005,0.009003
weth2band2weth_uniswap2Sushiswap,5,0.012590,0.044953,0.008991
weth2kp3rweth_sushiswap2Uniswap,18,0.038070,0.151708,0.008428
weth2sushi2weth_sushiswap2Uniswap,4,0.013420,0.020700,0.005175
weth2sushi2weth_uniswap2Sushiswap,8,0.006488,0.024930,0.003116


In [10]:
df2 = pd.DataFrame(pd.read_csv("../graveyard/arbbot_ver1.csv"))

In [19]:
df2.sort_values(by="netProfit", ascending=0).head(10)

NameError: name 'df2' is not defined